In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pandas as pd
import re
from datetime import date, timedelta
import time

In [4]:
def get_weather(sdate, edate):

    names = ['Date', 'Team 1', 'Team 2', 'Team 1 Score', 'Team 2 Score', 'Venue', 'Weather (Degrees)', 'Weather Type', 'Wind (MPH)', 'Wind Direction']
    df = pd.DataFrame(columns = names)  

    delta = edate - sdate

    failures = []

    count = 0

    driver = webdriver.Chrome(executable_path = 'C:\Drivers\Chrome_driver\chromedriver')

    for i in range(delta.days + 1):
        links = []
        day = sdate + timedelta(days=i)
        url = 'https://www.mlb.com/scores/' + str(day)
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser') 
        boxes = soup.find_all('a', class_ = re.compile('trk-box'))
        for box in boxes:
            if str(box.parent.parent.parent).find('czAVYt') == -1:
                links.append(box['href'])
        for link in links:
            driver.get(link)
            time.sleep(3)
            driver.maximize_window()
            data = driver.page_source
            soup = BeautifulSoup(data, 'html.parser')
            try:
                header = soup.find('div', class_ = 'header')
                header = header.text
                header = re.split(', | ', header)
                if not header[1].isdigit():
                    header[0 : 2] = [' '.join(header[0 : 2])] 
                if not header[-3].isdigit():
                    header[-3 : -1] = [' '.join(header[-3 : -1])]
                venue = soup.find('span', text = 'Venue').parent
                venue = venue.text
                venue = venue.replace('Venue: ', '')
                venue = venue.replace('.', '')
                weather = soup.find('span', text = 'Weather').parent
                weather = weather.text
                weather = weather.replace('Weather: ', '')
                weather = weather.replace(' degrees', '')
                weather = weather.replace('.', '')
                weather = weather.split(', ')
                wind = soup.find('span', text = 'Wind').parent
                wind = wind.text
                wind = wind.replace('Wind: ', '')
                wind = wind.replace(' mph', '')
                wind = wind.replace('.', '')
                wind = wind.split(', ')
                df.at[count, 'Date'] = day
                df.at[count, 'Team 1'] = header[2]
                df.at[count, 'Team 2'] = header[0]
                df.at[count, 'Team 1 Score'] = header[3] 
                df.at[count, 'Team 2 Score'] = header[1]
                df.at[count, 'Venue'] = venue
                df.at[count, 'Weather (Degrees)'] = weather[0]
                df.at[count, 'Weather Type'] = weather[1]
                df.at[count, 'Wind (MPH)'] = wind[0]
                df.at[count, 'Wind Direction'] = wind[1]
                count += 1
            except:
                failures.append(link)
    driver.close()
    #df.to_csv('weather.csv', index = False)
    return df

In [6]:
df = get_weather(date(2014, 3, 22), date(2019, 9, 29))

[]

In [ ]:
failures

In [21]:
df

,Date,Home,Away,Home Score,Away Score,Venue,Weather (Degrees),Weather Type,Wind (MPH),Wind Direction
0,2014-03-22,D-backs,Dodgers,1,3,Sydney Cricket Ground,75,Overcast,5,Varies
1,2014-03-22,D-backs,Dodgers,5,7,Sydney Cricket Ground,79,Partly Cloudy,5,Varies
2,2014-03-30,Dodgers,Padres,1,3,Petco Park,67,Clear,11,L To R
3,2014-03-31,Cubs,Pirates,0,1,PNC Park,54,Sunny,2,Out To CF
4,2014-03-31,Royals,Tigers,3,4,Comerica Park,52,Sunny,4,In From CF
